In [ ]:
#load vodka data
import pandas as pd
from sqlalchemy import create_engine
import pyodbc
# Database connection details
db_user = 'postgres'
db_password = 'password'
db_host = 'localhost'
db_port = '5432'
db_name = 'lca'
schema_name = 'PED'  # Specify schema here

# Create a connection string to postgres
postgres_conn = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
postgres_local = create_engine(postgres_conn)

azure_conn = "Driver={ODBC Driver 18 for SQL Server};Server=tcp:<hostname>.database.windows.net,1433;Database=<dbname>;Uid=<username>;Pwd=<super secret password>;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
# Create an engine object
azure = create_engine(f"mssql+pyodbc:///?odbc_connect={azure_conn}")

# Define your SQL query
query = 'SELECT * FROM "PED".vodka_sales'

# Read data in chunks
chunksize = 100000  # Number of rows per chunk
counter=1
for chunk in pd.read_sql(query, postgres_local, chunksize=chunksize):
    # Process each chunk
    print("chunk#= ",counter,"aprox:",counter*chunksize)
    # Load DataFrame to Azure SQL with schema
    chunk.to_sql('vodka_sales', azure, schema='dbo', if_exists='append', index=False)
    counter+=1
    
print("File loaded to Azure SQL DB successfully!")